In [1]:
import sys, platform, torch
import transformers
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import accelerate
import os
import re
import logging
from tqdm import tqdm
import pandas as pd

In [2]:
"""
pkill -u "$USER" -f python
if memory is insufficient
"""

'\npkill -u "$USER" -f python\nif memory is insufficient\n'

In [3]:
CACHE_DIR = "./models"          # download/cache here
INPUT_CSV = "test.csv"          # expects columns: ID, Question
OUTPUT_CSV = "beomi_polyglot_9_one.csv"   # submission format: ID, Answer

In [4]:
# model_id = "snunlp/KR-Medium"  # GPT2-like Korean LM
# model_id = "EleutherAI/polyglot-ko-5.8b"  # Mistral-based KULLM successor (wasn't very good)
model_id = "beomi/KoAlpaca-Polyglot-12.8B"

tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir="./models", padding_side="left")

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,                 # or load_in_8bit=True
    device_map="auto",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
)

device = "cuda" if torch.cuda.is_available() else "cpu"

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

In [5]:
def build_prompt(question):
    return f"""질문에 답하세요.

규칙:
1. 먼저 질문이 객관식인지 주관식인지 판단하세요.

2. 객관식일 경우:
   - 우선 선택지를 분석해 질문 범위에서 벗어나는 항목들을 찾아 제거하세요.
   - 남은 항목의 번호의 (예: 1, 2, 3, 4, 5) 하나를 "답:" 이후에 출력하세요.
   - 답은 하나밖에 없습니다

3. 주관식일 경우:
   - 질문의 핵심 키워드를 모두 포함하여 구체적으로 답변하세요.
   - 불필요한 서두, 결론, 접속사는 쓰지 마세요
   - 전문 용어는 표준 형태로 사용하세요.
   - 의미 유사도와 키워드 재현율을 높이기 위해 3~6개의 핵심 용어를 반드시 포함하세요.
   - 링크를 넣지 마세요

---

[예시 시작]

문제:
금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?
1 소비자금융업 2 투자자문업 3 투자매매업 4 투자중개업 5 보험중개업
답:
5

문제:
트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표를 설명하세요.
답:
트로이 목마 기반 원격제어 악성코드(RAT)는 사용자 몰래 시스템에 설치되어 원격 제어를 가능하게 하며, 암호화된 통신을 통해 명령제어(C2) 서버와 데이터를 주고받습니다. 주로 백도어를 설치하여 취약점을 악용하고, 키로깅을 통해 비밀번호나 금융정보를 탈취합니다. 주요 탐지 지표로는 비정상적인 네트워크 트래픽, 의심스러운 프로세스 생성, 레지스트리 변경, 그리고 시스템 자원 사용량의 급격한 증가 등이 있으며, 이러한 행위기반 분석을 통해 탐지할 수 있습니다.

문제:
전자금융거래법에 따라 이용자가 금융 분쟁조정을 신청할 수 있는 기관을 기술하세요.
답:
금융감독원, 한국소비자원, 금융분쟁조정위원회

[예시 끝]

---

질문:
{question}

답:"""

In [6]:
def build_prompt_two(question):
    return f"""맨 마지막 질문에 답하세요.

규칙:
1. 먼저 질문이 객관식인지 주관식인지 판단하세요.

2. 객관식일 경우:
   - 질문에 ‘옳지 않은 것’, ‘해당하지 않는 것’, ‘틀린 것’ 등의 표현이 있으면 → 보기 중 **틀린 하나**를 고르세요.
   - 그렇지 않으면 → 보기 중 **맞는 하나**를 고르세요.
   - **출력 형식:** 첫 번째 토큰은 숫자(1~5)만 적으세요. 그 앞뒤에 다른 글자, 괄호, 기호를 넣지 마세요.
     - 예시: `3` 또는 `1`
   - "답:" 이후에 출력하세요.
   - 답은 하나밖에 없습니다

3. 주관식일 경우:
   - 질문의 핵심 키워드를 모두 포함하여 구체적으로 답변하세요.
   - 불필요한 서두, 결론, 접속사는 쓰지 마세요
   - 전문 용어는 표준 형태로 사용하세요.
   - 의미 유사도와 키워드 재현율을 높이기 위해 3~6개의 핵심 용어를 반드시 포함하세요.
   - 링크를 넣지 마세요

3. 출력에는 다른 질문이나 답변 예시를 절대 포함하지 마세요.

---

[예시 시작]
질문:
트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표를 설명하세요.
답:
트로이 목마 기반 원격제어 악성코드(RAT)는 사용자 몰래 시스템에 설치되어 원격 제어를 가능하게 하며, 암호화된 통신을 통해 명령제어(C2) 서버와 데이터를 주고받습니다. 주로 백도어를 설치하여 취약점을 악용하고, 키로깅을 통해 비밀번호나 금융정보를 탈취합니다. 주요 탐지 지표로는 비정상적인 네트워크 트래픽, 의심스러운 프로세스 생성, 레지스트리 변경, 그리고 시스템 자원 사용량의 급격한 증가 등이 있으며, 이러한 행위기반 분석을 통해 탐지할 수 있습니다.

질문:
전자금융거래법에 따라 이용자가 금융 분쟁조정을 신청할 수 있는 기관을 기술하세요.
답:
금융감독원, 한국소비자원, 금융분쟁조정위원회

질문: 금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?
1 소비자금융업
2 투자자문업
3 투자매매업
4 투자중개업
5 보험중개업
답: 5

[예시 끝]

---

위 예시의 형식을 따라 다음 질문에 답하세요:
질문: {question}
답:"""

In [7]:
# load dataset and set up logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

test_df = pd.read_csv(INPUT_CSV)
assert {"ID", "Question"}.issubset(test_df.columns), "test.csv must have columns: ID, Question"


In [8]:
import re

UNI_DIGIT_MAP = str.maketrans({
    "①":"1","②":"2","③":"3","④":"4","⑤":"5",
    "⑴":"1","⑵":"2","⑶":"3","⑷":"4","⑸":"5",
    "❶":"1","❷":"2","❸":"3","❹":"4","❺":"5",
    "１":"1","２":"2","３":"3","４":"4","５":"5",
})

def extract_answer_from_output(output_text: str) -> str:
    # 1) Slice after the LAST marker to avoid few-shot contamination
    for marker in ["답:", "답변:", "답 :", "답변 :"]:
        if marker in output_text:
            output_text = output_text.rsplit(marker, 1)[-1]
            print(output_text)
            break

    ans = output_text.strip()

    # 2) Strip code fences and quotes
    import re
    ans = re.sub(r"^```.*?\n|\n```$", "", ans, flags=re.DOTALL).strip()
    ans = ans.strip('"\'' ).strip('"')

    # 3) Normalize circled/fullwidth digits + whitespace
    ans = ans.translate(UNI_DIGIT_MAP)
    ans = re.sub(r"\s+", " ", ans).strip()

   # MCQ-style prefix? (handles "5", "5번", "5.", "(5)", "5 (" )
    m = re.match(r'^\s*\(?\s*([1-5])\s*\)?\s*(?:번|[.)]|[,，、:;~-]|[\'"“”‘’]|(?=\s)|$)', ans)
    if m:
        return m.group(1)

    # Otherwise: 주관식 → return as-is (optionally trim length)
    return ans.strip()

In [ ]:
import math

batch_size = 8  # tune: 4~16 depending on VRAM
prompts = [build_prompt_two(str(q)) for q in test_df["Question"]]

answers = []
model.eval()

for i in tqdm(range(0, len(prompts), batch_size)):
    batch_prompts = prompts[i:i+batch_size]

    enc = tokenizer(
        batch_prompts,
        return_tensors="pt",
        padding=True,           # pad to longest in batch
        truncation=True,
        max_length=1024
    )
    enc.pop("token_type_ids", None)
    enc = {k: v.to(model.device, non_blocking=True) for k, v in enc.items()}

    with torch.inference_mode():
        out = model.generate(
            **enc,
            max_new_tokens=200,         
            do_sample=True,
            no_repeat_ngram_size=3,
            repetition_penalty=1.05,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=True,             # (default True) keep it on
            num_beams=1                 # beam search is slower
        )

    decoded = tokenizer.batch_decode(out, skip_special_tokens=True)

    print(decoded)
    answers.extend(decoded)

  2%|▏         | 1/65 [00:42<45:16, 42.45s/it]

['질문에 답하세요.\n\n규칙:\n1. 먼저 질문이 객관식인지 주관식인지 판단하세요.\n\n2. 객관식일 경우:\n   - 우선 선택지를 분석해 질문 범위에서 벗어나는 항목들을 찾아 제거하세요.\n   - 남은 항목의 번호의 (예: 1, 2, 3, 4, 5) 하나를 "답:" 이후에 출력하세요.\n   - 답은 하나밖에 없습니다\n\n3. 주관식일 경우:\n   - 질문의 핵심 키워드를 모두 포함하여 구체적으로 답변하세요.\n   - 불필요한 서두, 결론, 접속사는 쓰지 마세요\n   - 전문 용어는 표준 형태로 사용하세요.\n   - 의미 유사도와 키워드 재현율을 높이기 위해 3~6개의 핵심 용어를 반드시 포함하세요.\n   - 링크를 넣지 마세요\n\n---\n\n[예시 시작]\n\n문제:\n금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?\n1 소비자금융업 2 투자자문업 3 투자매매업 4 투자중개업 5 보험중개업\n답:\n5\n\n문제:\n트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표를 설명하세요.\n답:\n트로이 목마 기반 원격제어 악성코드(RAT)는 사용자 몰래 시스템에 설치되어 원격 제어를 가능하게 하며, 암호화된 통신을 통해 명령제어(C2) 서버와 데이터를 주고받습니다. 주로 백도어를 설치하여 취약점을 악용하고, 키로깅을 통해 비밀번호나 금융정보를 탈취합니다. 주요 탐지 지표로는 비정상적인 네트워크 트래픽, 의심스러운 프로세스 생성, 레지스트리 변경, 그리고 시스템 자원 사용량의 급격한 증가 등이 있으며, 이러한 행위기반 분석을 통해 탐지할 수 있습니다.\n\n문제:\n전자금융거래법에 따라 이용자가 금융 분쟁조정을 신청할 수 있는 기관을 기술하세요.\n답:\n금융감독원, 한국소비자원, 금융분쟁조정위원회\n\n[예시 끝]\n\n---\n\n질문:\n금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?\n1 소비자금융업\n2 투자자문업\n3 투자매매업\n4 투자중개업\n5 보험중개업

  3%|▎         | 2/65 [01:24<44:36, 42.49s/it]

['질문에 답하세요.\n\n규칙:\n1. 먼저 질문이 객관식인지 주관식인지 판단하세요.\n\n2. 객관식일 경우:\n   - 우선 선택지를 분석해 질문 범위에서 벗어나는 항목들을 찾아 제거하세요.\n   - 남은 항목의 번호의 (예: 1, 2, 3, 4, 5) 하나를 "답:" 이후에 출력하세요.\n   - 답은 하나밖에 없습니다\n\n3. 주관식일 경우:\n   - 질문의 핵심 키워드를 모두 포함하여 구체적으로 답변하세요.\n   - 불필요한 서두, 결론, 접속사는 쓰지 마세요\n   - 전문 용어는 표준 형태로 사용하세요.\n   - 의미 유사도와 키워드 재현율을 높이기 위해 3~6개의 핵심 용어를 반드시 포함하세요.\n   - 링크를 넣지 마세요\n\n---\n\n[예시 시작]\n\n문제:\n금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?\n1 소비자금융업 2 투자자문업 3 투자매매업 4 투자중개업 5 보험중개업\n답:\n5\n\n문제:\n트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표를 설명하세요.\n답:\n트로이 목마 기반 원격제어 악성코드(RAT)는 사용자 몰래 시스템에 설치되어 원격 제어를 가능하게 하며, 암호화된 통신을 통해 명령제어(C2) 서버와 데이터를 주고받습니다. 주로 백도어를 설치하여 취약점을 악용하고, 키로깅을 통해 비밀번호나 금융정보를 탈취합니다. 주요 탐지 지표로는 비정상적인 네트워크 트래픽, 의심스러운 프로세스 생성, 레지스트리 변경, 그리고 시스템 자원 사용량의 급격한 증가 등이 있으며, 이러한 행위기반 분석을 통해 탐지할 수 있습니다.\n\n문제:\n전자금융거래법에 따라 이용자가 금융 분쟁조정을 신청할 수 있는 기관을 기술하세요.\n답:\n금융감독원, 한국소비자원, 금융분쟁조정위원회\n\n[예시 끝]\n\n---\n\n질문:\n금융권에서 SBOM을 활용하는 이유로 가장 적절한 것은?\n1 금융 시스템의 접근 제어 정책을 효율적으로 구현하기 위해\n2 금융 거래의 투명성을 높

  5%|▍         | 3/65 [02:07<43:52, 42.46s/it]

...

['질문에 답하세요.\n\n규칙:\n1. 먼저 질문이 객관식인지 주관식인지 판단하세요.\n\n2. 객관식일 경우:\n   - 우선 선택지를 분석해 질문 범위에서 벗어나는 항목들을 찾아 제거하세요.\n   - 남은 항목의 번호의 (예: 1, 2, 3, 4, 5) 하나를 "답:" 이후에 출력하세요.\n   - 답은 하나밖에 없습니다\n\n3. 주관식일 경우:\n   - 질문의 핵심 키워드를 모두 포함하여 구체적으로 답변하세요.\n   - 불필요한 서두, 결론, 접속사는 쓰지 마세요\n   - 전문 용어는 표준 형태로 사용하세요.\n   - 의미 유사도와 키워드 재현율을 높이기 위해 3~6개의 핵심 용어를 반드시 포함하세요.\n   - 링크를 넣지 마세요\n\n---\n\n[예시 시작]\n\n문제:\n금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?\n1 소비자금융업 2 투자자문업 3 투자매매업 4 투자중개업 5 보험중개업\n답:\n5\n\n문제:\n트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표를 설명하세요.\n답:\n트로이 목마 기반 원격제어 악성코드(RAT)는 사용자 몰래 시스템에 설치되어 원격 제어를 가능하게 하며, 암호화된 통신을 통해 명령제어(C2) 서버와 데이터를 주고받습니다. 주로 백도어를 설치하여 취약점을 악용하고, 키로깅을 통해 비밀번호나 금융정보를 탈취합니다. 주요 탐지 지표로는 비정상적인 네트워크 트래픽, 의심스러운 프로세스 생성, 레지스트리 변경, 그리고 시스템 자원 사용량의 급격한 증가 등이 있으며, 이러한 행위기반 분석을 통해 탐지할 수 있습니다.\n\n문제:\n전자금융거래법에 따라 이용자가 금융 분쟁조정을 신청할 수 있는 기관을 기술하세요.\n답:\n금융감독원, 한국소비자원, 금융분쟁조정위원회\n\n[예시 끝]\n\n---\n\n질문:\n책임감 있는 AI(Responsible AI)의 구현을 위해 금융권에서 고려해야 할 요소로 가장 적절한 것은?\n1 AI의 비용 절감\n2 AI의 신

 98%|█████████▊| 64/65 [44:46<00:42, 42.51s/it]

['질문에 답하세요.\n\n규칙:\n1. 먼저 질문이 객관식인지 주관식인지 판단하세요.\n\n2. 객관식일 경우:\n   - 우선 선택지를 분석해 질문 범위에서 벗어나는 항목들을 찾아 제거하세요.\n   - 남은 항목의 번호의 (예: 1, 2, 3, 4, 5) 하나를 "답:" 이후에 출력하세요.\n   - 답은 하나밖에 없습니다\n\n3. 주관식일 경우:\n   - 질문의 핵심 키워드를 모두 포함하여 구체적으로 답변하세요.\n   - 불필요한 서두, 결론, 접속사는 쓰지 마세요\n   - 전문 용어는 표준 형태로 사용하세요.\n   - 의미 유사도와 키워드 재현율을 높이기 위해 3~6개의 핵심 용어를 반드시 포함하세요.\n   - 링크를 넣지 마세요\n\n---\n\n[예시 시작]\n\n문제:\n금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?\n1 소비자금융업 2 투자자문업 3 투자매매업 4 투자중개업 5 보험중개업\n답:\n5\n\n문제:\n트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표를 설명하세요.\n답:\n트로이 목마 기반 원격제어 악성코드(RAT)는 사용자 몰래 시스템에 설치되어 원격 제어를 가능하게 하며, 암호화된 통신을 통해 명령제어(C2) 서버와 데이터를 주고받습니다. 주로 백도어를 설치하여 취약점을 악용하고, 키로깅을 통해 비밀번호나 금융정보를 탈취합니다. 주요 탐지 지표로는 비정상적인 네트워크 트래픽, 의심스러운 프로세스 생성, 레지스트리 변경, 그리고 시스템 자원 사용량의 급격한 증가 등이 있으며, 이러한 행위기반 분석을 통해 탐지할 수 있습니다.\n\n문제:\n전자금융거래법에 따라 이용자가 금융 분쟁조정을 신청할 수 있는 기관을 기술하세요.\n답:\n금융감독원, 한국소비자원, 금융분쟁조정위원회\n\n[예시 끝]\n\n---\n\n질문:\n개인정보보호법 제12조에 따르면, 보호위원회가 개인정보처리자에게 권장할 수 있는 표준 개인정보 보호지침의 내용으로 적절한 것은?\n1 개인정보 유출 

100%|██████████| 65/65 [44:51<00:00, 41.40s/it]

['질문에 답하세요.\n\n규칙:\n1. 먼저 질문이 객관식인지 주관식인지 판단하세요.\n\n2. 객관식일 경우:\n   - 우선 선택지를 분석해 질문 범위에서 벗어나는 항목들을 찾아 제거하세요.\n   - 남은 항목의 번호의 (예: 1, 2, 3, 4, 5) 하나를 "답:" 이후에 출력하세요.\n   - 답은 하나밖에 없습니다\n\n3. 주관식일 경우:\n   - 질문의 핵심 키워드를 모두 포함하여 구체적으로 답변하세요.\n   - 불필요한 서두, 결론, 접속사는 쓰지 마세요\n   - 전문 용어는 표준 형태로 사용하세요.\n   - 의미 유사도와 키워드 재현율을 높이기 위해 3~6개의 핵심 용어를 반드시 포함하세요.\n   - 링크를 넣지 마세요\n\n---\n\n[예시 시작]\n\n문제:\n금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?\n1 소비자금융업 2 투자자문업 3 투자매매업 4 투자중개업 5 보험중개업\n답:\n5\n\n문제:\n트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표를 설명하세요.\n답:\n트로이 목마 기반 원격제어 악성코드(RAT)는 사용자 몰래 시스템에 설치되어 원격 제어를 가능하게 하며, 암호화된 통신을 통해 명령제어(C2) 서버와 데이터를 주고받습니다. 주로 백도어를 설치하여 취약점을 악용하고, 키로깅을 통해 비밀번호나 금융정보를 탈취합니다. 주요 탐지 지표로는 비정상적인 네트워크 트래픽, 의심스러운 프로세스 생성, 레지스트리 변경, 그리고 시스템 자원 사용량의 급격한 증가 등이 있으며, 이러한 행위기반 분석을 통해 탐지할 수 있습니다.\n\n문제:\n전자금융거래법에 따라 이용자가 금융 분쟁조정을 신청할 수 있는 기관을 기술하세요.\n답:\n금융감독원, 한국소비자원, 금융분쟁조정위원회\n\n[예시 끝]\n\n---\n\n질문:\n하이브리드 위협에 대한 설명으로 가장 적절한 것은?\n1 사이버 공간에서만 발생하는 위협\n2 단일 채널을 통해 발생하는 위협\n3 온·오프라인 연계

In [10]:
ans = [extract_answer_from_output(text) for text in answers]

 글쎄... 뭔가 빠진 느낌이야...

-_-;
다시 해 보자.
금융업을 크게 다섯 분야로 나누고, 그 안에서 또 각각을 세분화해서 분류했다고 했지? 뭐가 더 있을 것 같은데...
헛, 가만있자... '은행, 증권, 보험, 신탁' 이 있었지? 그리고 '소비자, 생산자, 기업, 정부' 있었고...
그럼 이걸 어떻게 구분하냐고?
5. 은행 (Bank)
6. 투자매매업자 (Mutual Sales Office)
7. 투자중개업자 (Brokerage House)
8. 소비자금융 (Consumer Loans)
9. 보험 (Insurance)
10. 기타 금융업 (Others)
맞나? 틀릴래야 틀릴 수가 없는데...-_-?
그럼 은행과 보험을 먼저 볼까?
위험 관리 전략 수립시에는 위험 대응을 위한 대상, 범위, 유형, 영향도, 손실 모형, 복원력, 실패시 원인 및 영향도 등을 분석하여 적절한 대응 방안을 수립하여야 합니다. 또한, 위험 관리를 위한 조직 문화, 규제 및 법적 요구사항, 예산 및 프로젝트 관리자 교육 등도 고려하여야 하며 이 모든 요소가 계획에 반영되어야 합니다.
P.S 답변이 도움 되셨길 바랍니다^^

### 답변:Risk 관리는 조직 전반에 걸친 매우 중요한 문제입니다. 위험 관리 계획에는 주요 손실 발생 지점, 위협 요인, 위험 수준, 대응 전략, 조직 문화 등이 포함되어야 하며 모든 부서가 이해하고 실행할 수있는 실질적인 조치가 담겨있어야 합니다." 

 고려사항에는 다음과 같은 것들이 포함될 수 있으며 반드시 모든 요소를 다 포함해야 합니다:
 - (1) 위험
 5
 4번 목표 시간 정의는 계획의 일부일 뿐이며, 가장 중요한 것은 복구 절차와 방법, 그리고 인력과 장비 등을 계획하여 인명피해를 최소화하고 경제적인 피해를 최대화하는 것입니다.
추가 내용:
- 화재, 태풍, 지진 등 자연재해와 인재는 평소와 다른 양상으로 발생할 가능성이 높아, 발생 가능한 모든 시나리오를 계획하여 대처할 필요가 있습니다.
- 중요한 데이터가 있다면 백업을 해두는 것이

In [11]:
ans

["글쎄... 뭔가 빠진 느낌이야... -_-; 다시 해 보자. 금융업을 크게 다섯 분야로 나누고, 그 안에서 또 각각을 세분화해서 분류했다고 했지? 뭐가 더 있을 것 같은데... 헛, 가만있자... '은행, 증권, 보험, 신탁' 이 있었지? 그리고 '소비자, 생산자, 기업, 정부' 있었고... 그럼 이걸 어떻게 구분하냐고? 5. 은행 (Bank) 6. 투자매매업자 (Mutual Sales Office) 7. 투자중개업자 (Brokerage House) 8. 소비자금융 (Consumer Loans) 9. 보험 (Insurance) 10. 기타 금융업 (Others) 맞나? 틀릴래야 틀릴 수가 없는데...-_-? 그럼 은행과 보험을 먼저 볼까?",
 '위험 관리 전략 수립시에는 위험 대응을 위한 대상, 범위, 유형, 영향도, 손실 모형, 복원력, 실패시 원인 및 영향도 등을 분석하여 적절한 대응 방안을 수립하여야 합니다. 또한, 위험 관리를 위한 조직 문화, 규제 및 법적 요구사항, 예산 및 프로젝트 관리자 교육 등도 고려하여야 하며 이 모든 요소가 계획에 반영되어야 합니다. P.S 답변이 도움 되셨길 바랍니다^^ ### 답변:Risk 관리는 조직 전반에 걸친 매우 중요한 문제입니다. 위험 관리 계획에는 주요 손실 발생 지점, 위협 요인, 위험 수준, 대응 전략, 조직 문화 등이 포함되어야 하며 모든 부서가 이해하고 실행할 수있는 실질적인 조치가 담겨있어야 합니다." 고려사항에는 다음과 같은 것들이 포함될 수 있으며 반드시 모든 요소를 다 포함해야 합니다: - (1) 위험',
 '5',
 '4',
 'RAT은 사용자 몰래시스템에 설치됩니다. 트로이 목마다. 큰 카테고리로 사용자이름과 그밖의 정보를 탈취하거나 컴퓨터를 원격 조종하기 위한 수단으로 쓰입니다. 주요 탐지로키로깅을 통한 비밀번호 탈취,악성코드 제작자가 저장해놓은 기밀정보가 있습니다주요 방어기술은실시간 예방, 이상 징후 감지 및 대응,샌드박스, 가상머신, 다중요소 인증, 네트워크 보안 등이 잇습니다.그러

In [12]:
submission = pd.DataFrame({
    "ID": test_df["ID"],
    "Answer": ans
})

submission.to_csv(OUTPUT_CSV, index=False)